In [ ]:
USE_SIMULATOR = False

# Circuit Parameter Values
EGO_CIRCUIT_PREPARATION_PARAMETERS = {
    1:  [3.85532],
    2:  [3.28449, 3.73368],
    3:  [3.11251, 3.3066,  3.72502],
    4:  [3.14757, 3.09809, 3.31778, 3.72279], 
    5:  [3.14036, 3.15256, 3.09041, 3.3243,  3.72197],
    6:  [3.14185, 3.13892, 3.15581, 3.08549, 3.32861, -2.56156],
    7:  [3.14154, 3.14223, 3.13779, 3.15815, 3.08202, 3.33168, -2.56169],
    8:  [3.1416,  3.14144, 3.14258, 3.13688, 3.15993, 3.07943, 3.33399, -2.56173],
    9:  [3.14159, 3.14163, 3.14134, 3.1429,  3.13613, 3.16133, 3.07742, 3.3358, -2.56172],
    10: [3.14159, 3.14158, 3.14166, 3.14124, 3.14317, 3.13551, 3.16247, 3.07582, 3.33725, -2.5617],
    11: [3.14159, 3.14159, 3.14158, 3.14169, 3.14115, 3.14341, 3.13499, 3.16341, 3.0745,   3.33844, -2.56166],
}

In [ ]:
# Ansatz Circuit Construction
import jaqalpaq as jql
from qscout.v1.std.jaqal_gates import ALL_GATES
import numpy as np

def _prepare_state(
    circuit_parameters,
    number_of_qubits,
    circuit_builder,
):
    """Constructs the circuit to prepare the initial state that is used during measurement of all cliques"""
    assert len(circuit_parameters) == number_of_qubits
    assert number_of_qubits > 0
    
    # The reduced-unarily-encoded EGO circuit
    qubits = circuit_builder.register("q", number_of_qubits)
    circuit_builder.gate("prepare_all")
        
    
    theta0 = circuit_builder.let("theta0", circuit_parameters[0])
    circuit_builder.gate("Ry", qubits[0], theta0)


    for control_qubit_index in range(0, number_of_qubits-1):
        control_qubit = qubits[control_qubit_index]
        target_qubit = qubits[control_qubit_index + 1]
        theta_value = circuit_parameters[control_qubit_index+1]
        
        positive_halved_theta = circuit_builder.let("h_theta{}".format(control_qubit_index+1), theta_value/2)
        negative_halved_theta = circuit_builder.let("nh_theta{}".format(control_qubit_index+1), -theta_value/2)
        
        circuit_builder = _add_controlled_ry_gate(circuit_builder, control_qubit, target_qubit,
                                                  positive_halved_theta, negative_halved_theta)
        
    return circuit_builder, qubits

def _add_cnot_gate(circuit_builder, control, target):
    """CNOT Gate from Maslov (2017)"""
    circuit_builder.gate("Sy", control)
    circuit_builder.gate("Sxx", control, target)
    circuit_builder.gate("Sxd", control)
    circuit_builder.gate("Sxd", target)
    circuit_builder.gate("Syd", control)
    return circuit_builder


def _add_hadamard_gate(circuit_builder, qubit):
    """Hadamard Gate from JAQAL (https://www.sandia.gov/quantum/Projects/quantum_assembly_spec.pdf)"""
    circuit_builder.gate("Sy", qubit)
    circuit_builder.gate("Px", qubit)
    return circuit_builder

def _add_controlled_ry_gate(circuit_builder, control, target,
                            positive_halved_theta, negative_halved_theta):
    #     Optimized Gate Decomposition for CRy
    circuit_builder.gate("Rz", target, -np.pi/2)
    circuit_builder.gate("Rx", target, positive_halved_theta)
    circuit_builder.gate("Rx", control, np.pi/2)
    circuit_builder.gate("Rz", control, np.pi/2)
    circuit_builder.gate("MS", control, target, 0, negative_halved_theta)
    circuit_builder.gate("Rz", target, np.pi/2)
    circuit_builder.gate("Rz", control, -np.pi/2)
    circuit_builder.gate("Rx", control, -np.pi/2)
    return circuit_builder

In [ ]:
def create_clique1_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 1"""
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)

    builder, _ = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )
    builder.gate("measure_all")
    return builder.build()


def create_clique2_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 2"""
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)
    
    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )
    
    # Hadamard rotation on all qubits to change measurement context
    for i in range(number_of_qubits):
        builder = _add_hadamard_gate(builder, qubits[i])
    builder.gate("measure_all")
    return builder.build()


def create_clique3_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 3"""
    assert number_of_qubits > 1
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)

    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )

    builder = _add_hadamard_gate(builder, qubits[1])
    builder = _add_cnot_gate(builder, qubits[0], qubits[1])
    builder = _add_hadamard_gate(builder, qubits[0])

    builder.gate("measure_all")
    return builder.build()


def create_clique4_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 4"""
    assert number_of_qubits > 2
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)
    
    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )

    builder = _add_hadamard_gate(builder, qubits[2])
    builder = _add_cnot_gate(builder, qubits[1], qubits[2])
    builder = _add_hadamard_gate(builder, qubits[1])

    builder.gate("measure_all")
    return builder.build()

In [ ]:
from jaqalpaq.generator import generate_jaqal_program
from collections import Counter
from jaqalpaq import run

if USE_SIMULATOR:
    from qscout.v1.std.ionsim import IonSimErrorModel
    jql.core.result.ProbabilisticSubcircuit.CUTOFF_FAIL = 1e-4
    jql.core.result.ProbabilisticSubcircuit.CUTOFF_WARN = 1e-4


def get_bitstrings_from_measured_probability_distribution(
    number_of_samples, probability_distribution
):
    number_of_qubits = int(np.log2(len(probability_distribution)))
    bitstrings = [
        format(i, "0{}b".format(number_of_qubits))[::-1]
        for i in range(2 ** number_of_qubits)]

    measurements = np.random.choice(bitstrings, size=number_of_samples, replace=True, p=probability_distribution)
    
    return measurements.tolist()

def run_circuit_and_get_probability_distribution_on_simulator(
    circuit, ideal=False, number_of_samples=1000
):
    probability_distribution = jql.emulator.run_jaqal_circuit(circuit).subcircuits[0].probability_by_int
    if ideal:
        return probability_distribution
    
    measurements = get_bitstrings_from_measured_probability_distribution(number_of_samples, probability_distribution)
    counts = Counter(measurements)
    
    number_of_qubits = int(np.log2(len(probability_distribution)))
    measured_probability_distribution = np.zeros(len(probability_distribution))
    for integer in range(len(probability_distribution)):
        binary = format(integer, '#0{}b'.format(number_of_qubits+2))[2:][::-1]
        measured_probability_distribution[integer] = counts[binary] / number_of_samples
    return measured_probability_distribution.tolist()

def run_circuit_and_get_probability_distribution_on_hardware(
    circuit,
):
    jaqal_program = "\n".join(["from Calibration_PulseDefinitions.CalibrationPulses usepulses *"] + [generate_jaqal_program(circuit)])
    jaqal_program = jaqal_program.replace('register q[1]','register q[3]')
    jaqal_program = jaqal_program.replace('register q[2]', 'register q[3]')
    
    experiment_result = run.run_jaqal_circuit(jaqal_program)
    probability_distribution = experiment_result.subcircuits[0].relative_frequency_by_int
    
#     if number_of_qubits == 1:
#         probability_distribution = np.asarray([[sum(probability_distribution[::2]),sum(probability_distribution[1::2])]])
#     elif number_of_qubits == 2:
#         probability_distribution = np.asarray([(np.array(probability_distribution[:1<<(number_of_qubits)]) + np.array(probability_distribution[1<<(number_of_qubits):])).tolist()])
#     else:
#         probability_distribution = np.asarray([probability_distribution])
    
    return probability_distribution.tolist()

In [ ]:
from scipy.special import rel_entr
import copy

def calculate_kl_divergence(ideal_distribution, measured_distribution):
    ideal_distribution = copy.deepcopy(ideal_distribution)
    measured_distribution = copy.deepcopy(measured_distribution)
    
    for distribution in [ideal_distribution, measured_distribution]:
        for i, _ in enumerate(distribution):
            if distribution[i] == 0:
                distribution[i] = 1e-32

    try:
        kl_divergence = sum(rel_entr(ideal_distribution, measured_distribution))
    except ValueError as e:
        kl_divergence = -1
        
    return kl_divergence

In [ ]:
# Run LMG EGO Circuit to Prepare Ground States for Different Problem Sizes (Number of Qubits)
import json
import os
import datetime

NUMBER_OF_QUBITS = 3

date = datetime.datetime.now()

device = "hardware"
if USE_SIMULATOR:
    device = "simulator"
RESULTS_DIRECTORY = "../results/point/{}_{}-{}-{}/".format(device, date.month, date.day, date.year)

if not os.path.exists(RESULTS_DIRECTORY):
    os.makedirs(RESULTS_DIRECTORY)

NUMBER_OF_SAMPLES = 10000
    
circuits = []
parameters = []
for parameter in EGO_CIRCUIT_PREPARATION_PARAMETERS[NUMBER_OF_QUBITS]:
    # Fit parameters to -pi -> pi range
    if parameter > np.pi:
        parameter -= 2*np.pi
    parameters.append(parameter)

# Determine cliques (circuits) to measure for experiment with M qubits
circuits.append(create_clique1_circuit(parameters, NUMBER_OF_QUBITS))
circuits.append(create_clique2_circuit(parameters, NUMBER_OF_QUBITS))
if NUMBER_OF_QUBITS > 1:
    circuits.append(create_clique3_circuit(parameters, NUMBER_OF_QUBITS))
if NUMBER_OF_QUBITS > 2:
    circuits.append(create_clique4_circuit(parameters, NUMBER_OF_QUBITS))


run_circuit = run_circuit_and_get_probability_distribution_on_hardware
if USE_SIMULATOR:
    run_circuit = run_circuit_and_get_probability_distribution_on_simulator

print("\nKL Divergences for {} Qubits:".format(NUMBER_OF_QUBITS))
probability_distributions = []
for clique_index, circuit in enumerate(circuits):
    probability_distribution = run_circuit(circuit)
    probability_distributions.append(probability_distribution)
    ideal_distribution = run_circuit_and_get_probability_distribution_on_simulator(circuit, ideal=True)
    # relative entropy of measured distribution from ideal
    kl_divergence = calculate_kl_divergence(ideal_distribution, probability_distribution)
    if kl_divergence < 0.:
        print("    Clique {} is: Error Calculating KL Divergence for Distributions with Differing Lengths".format(clique_index))
    else:
        print("    Clique {} is: ".format(clique_index), kl_divergence)
        
data_filename = RESULTS_DIRECTORY + "probability_data_{}_qubits.json".format(NUMBER_OF_QUBITS)
with open(data_filename, "w") as f:
    f.write(json.dumps(probability_distributions))
f.close()

measurements = [
    get_bitstrings_from_measured_probability_distribution(
        NUMBER_OF_SAMPLES, probability_distribution
    )
    for probability_distribution in probability_distributions
]

measurement_data_filename = RESULTS_DIRECTORY + "measurement_data_{}_qubits.json".format(NUMBER_OF_QUBITS)

with open(measurement_data_filename, "w") as f:
    f.write(json.dumps(measurements))
f.close()
